In [1]:
import numpy as np
x = np.fromfile("bin/pre-comp.bin", np.complex64)
a = np.abs(x)
print("max|x| =", a.max())
print("p99|x| =", np.quantile(a, 0.99))
print("rms|x| =", np.sqrt(np.mean(a*a)))


max|x| = 3.1618295
p99|x| = 2.835333204269409
rms|x| = 1.9996134


In [10]:
import numpy as np
x = np.fromfile("bin/pre-comp.bin", np.complex64)
y = np.fromfile("bin/post-comp.bin", np.complex64)
n = min(len(x), len(y))
e = y[:n] - x[:n]
print("NMSE =", np.mean(np.abs(e)**2) / np.mean(np.abs(x[:n])**2))
print("EVM_rms =", np.sqrt(np.mean(np.abs(e)**2) / np.mean(np.abs(x[:n])**2)))


NMSE = 0.0015020209
EVM_rms = 0.038755912
